**Youssef Mohammed Abd El-Hai : 20200669**

**Rina Khaled Ahmed Mortada   : 20201078**

**Rawan Khaled Abd El-Monem   : 20201072**

In [ ]:
import sys
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
def predict(x, w, b):
    y_pred = sigmoid(np.dot(x, w) + b)

    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0

    return y_pred

In [ ]:
def calculate_y_pred_regularized(x, w, b, lam = 0.1):
    y_pred = sigmoid(np.dot(x, w) + b  + lam * np.sum(np.abs(w)))

    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0

    return y_pred

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def standardize(data):
  return (data - np.mean(data)) / (np.std(data) + sys.float_info.min)

In [ ]:
def accuracy(y, y_hat):
    return np.mean(y == y_hat)

In [ ]:
def Logistic_Regression(x, y, learning_rate = 0.1, iterations = 100):

    w = random.rand(x.shape[1])
    b = random.rand(1)

    tolerance=0.00001

    for i in range(iterations):
        y_pred = predict(x, w, b)

        if(np.mean((y-y_pred)**2)<tolerance):
          break

        dw = np.dot(x.T, (y_pred - y)) / len(x)
        db = np.mean(y_pred - y)

        w -= learning_rate * dw
        b -= learning_rate * db

    return w,b

In [ ]:
def Logistic_Regression_Regularized(x, y, learning_rate = 0.1, iterations = 100, lam = 0.1):

    w = random.rand(x.shape[1])
    b = random.rand(1)

    tolerance=0.00001

    for i in range(iterations):

        y_pred = calculate_y_pred_regularized(x, w, b)

        if(np.mean((y-y_pred)**2)<tolerance):
            break

        dw = (np.dot(x.T, (y_pred - y)) + np.where(w>0, lam, -lam) / len(x))
        db = np.mean(y_pred - y)

        w -= learning_rate * dw
        b -= learning_rate * db

    return w,b

In [ ]:
def Log_Reg_Regularized_minibatch(x, y, learning_rate = 0.1, iterations = 100, lam = 0.1, batchsize=64):

    w = random.rand(x.shape[1])
    b = random.rand(1)

    tolerance=0.00001

    for i in range(iterations):

        for j in range(int(x.shape[0]/batchsize)):

            if((j+1)*batchsize > x.shape[0]):
                xtrain = x[j*batchsize:]
                ytrain = y[j*batchsize:]
            else:
                xtrain = x[j*batchsize:(j+1)*batchsize]
                ytrain = y[j*batchsize:(j+1)*batchsize]

            y_pred = calculate_y_pred_regularized(xtrain, w, b)

            dw = (np.dot(xtrain.T, (y_pred - ytrain)) + np.where(w>0, lam, -lam) / len(x))
            db = np.mean(y_pred - ytrain)

            w -= learning_rate * dw
            b -= learning_rate * db

            # Testing the effect after each batch
            y_pred = calculate_y_pred_regularized(x, w, b)
            if(np.mean((y-y_pred)**2)<tolerance):
                return w,b

        y_pred = calculate_y_pred_regularized(x, w, b)
        if(np.mean((y-y_pred)**2)<tolerance):
            break

    return w,b

In [ ]:
def Log_Reg_Regularized_minibatch_using_RMS_Prop_optimzer(x, y, learning_rate = 0.1, iterations = 100, lam = 0.1, batchsize=64, beta=0.999, epcilon = 1e-8):

    w = random.rand(x.shape[1])
    b = random.rand(1)

    # Initialize Vdw for RMS_Prop optimizer
    Vdw = 0
    Vdb = 0

    t = 1

    tolerance=0.00001

    for i in range(iterations):

        for j in range(int(x.shape[0]/batchsize)):

            if((j+1)*batchsize > x.shape[0]):
                xtrain = x[j*batchsize:]
                ytrain = y[j*batchsize:]
            else:
                xtrain = x[j*batchsize:(j+1)*batchsize]
                ytrain = y[j*batchsize:(j+1)*batchsize]

            y_pred = calculate_y_pred_regularized(xtrain, w, b)

            dw = (np.dot(xtrain.T, (y_pred - ytrain)) + np.where(w>0, lam, -lam) / len(x))
            db = np.mean(y_pred - ytrain)

            # Update Vdw & Vdb
            biased_Vdw = beta * Vdw + (1-beta) * (dw**2)
            unbiased_Vdw = Vdw / (1-(beta**t))

            biased_Vdb = beta * Vdb + (1-beta) * (db**2)
            unbiased_Vdb = Vdb / (1-(beta**t))

            # Update w & b
            w -= learning_rate * (dw / (np.sqrt(unbiased_Vdw) + epcilon))
            b -= learning_rate * (db / (np.sqrt(unbiased_Vdb) + epcilon))

            # Testing the effect after each batch
            y_pred = calculate_y_pred_regularized(x, w, b)
            if(np.mean((y-y_pred)**2)<tolerance):
                return w,b

            # Times of weight update
            t+=1

        y_pred = calculate_y_pred_regularized(x, w, b)
        if(np.mean((y-y_pred)**2)<tolerance):
            break

    return w,b

In [ ]:
def Log_Reg_Regularized_minibatch_using_Adam_optimzer(x, y, learning_rate = 0.1, iterations = 100, lam = 0.1, batchsize=64, beta=0.999, epcilon = 1e-8):

    w = random.rand(x.shape[1])
    b = random.rand(1)

    # Initialize Vdw for RMS_Prop optimizer
    Vdw = 0
    Vdb = 0

    t = 1

    tolerance=0.00001

    for i in range(iterations):

        for j in range(int(x.shape[0]/batchsize)):

            if((j+1)*batchsize > x.shape[0]):
                xtrain = x[j*batchsize:]
                ytrain = y[j*batchsize:]
            else:
                xtrain = x[j*batchsize:(j+1)*batchsize]
                ytrain = y[j*batchsize:(j+1)*batchsize]

            y_pred = calculate_y_pred_regularized(xtrain, w, b)

            dw = (np.dot(xtrain.T, (y_pred - ytrain)) + np.where(w>0, lam, -lam) / len(x))
            db = np.mean(y_pred - ytrain)

            # Update Vdw & Vdb
            biased_Vdw = beta * Vdw + (1-beta) * (dw**2)
            mdw= beta * Vdw + (1-beta) * (dw)
            unbiased_Vdw = Vdw / (1-(beta**t))
            unbiased_mdw=mdw / (1-(beta**t))

            biased_Vdb = beta * Vdb + (1-beta) * (db**2)
            mdb= beta * Vdb + (1-beta) * (db)
            unbiased_Vdb = Vdb / (1-(beta**t))
            unbiased_mdb=mdb / (1-(beta**t))

            # Update w & b
            w -= learning_rate * (unbiased_mdw / (np.sqrt(unbiased_Vdw) + epcilon))
            b -= learning_rate * (unbiased_mdb / (np.sqrt(unbiased_Vdb) + epcilon))

            # Testing the effect after each batch
            y_pred = calculate_y_pred_regularized(x, w, b)
            if(np.mean((y-y_pred)**2)<tolerance):
                return w,b

            # Times of weight update
            t+=1

        y_pred = calculate_y_pred_regularized(x, w, b)
        if(np.mean((y-y_pred)**2)<tolerance):
            break

    return w,b

In [ ]:
data = pd.read_csv("/content/sample_data/mnist_train_small.csv")

In [ ]:
xtrain = data[data['6'].isin([0, 1])].drop(columns=['6'])

In [ ]:
y_train = data[data['6'].isin([0, 1])]['6']

In [ ]:
x_train = standardize(xtrain)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
x_train, y_train = shuffle(x_train, y_train)

In [ ]:
validation_ratio = 0.2

x_validation = x_train[:int(validation_ratio * x_train.shape[0])]
y_validation = y_train[:int(validation_ratio * y_train.shape[0])]

x_train = x_train[int(validation_ratio * x_train.shape[0]):]
y_train = y_train[int(validation_ratio * y_train.shape[0]):]

In [ ]:
w1, b1 = Logistic_Regression_Regularized(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.1)

y_pred1 = calculate_y_pred_regularized(x_validation, w1, b1, lam=0.1)

acc1 = accuracy(y_validation, y_pred1)
acc1

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.990487514863258

In [ ]:
w2, b2 = Logistic_Regression_Regularized(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.3)

y_pred2 = calculate_y_pred_regularized(x_validation, w2, b2, lam=0.3)

acc2 = accuracy(y_validation, y_pred2)
acc2

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9476813317479191

In [ ]:
w3, b3 = Log_Reg_Regularized_minibatch(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.1, batchsize=150)

y_pred3 = calculate_y_pred_regularized(x_validation, w3, b3, lam=0.1)

acc3 = accuracy(y_validation, y_pred3)
acc3

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9916765755053508

In [ ]:
w4, b4 = Log_Reg_Regularized_minibatch_using_RMS_Prop_optimzer(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.1, batchsize=150)

y_pred4 = calculate_y_pred_regularized(x_validation, w4, b4, lam=0.1)

acc4 = accuracy(y_validation, y_pred4)
acc4

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9928656361474435

In [ ]:
w5, b5 = Log_Reg_Regularized_minibatch_using_Adam_optimzer(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.1, batchsize=150)

y_pred5 = calculate_y_pred_regularized(x_validation, w5, b5, lam=0.1)

acc5 = accuracy(y_validation, y_pred5)
acc5

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9821640903686087

In [ ]:
#diff batch size
w6, b6 = Log_Reg_Regularized_minibatch(x_train, y_train, learning_rate=0.1,  iterations=100, lam=0.1, batchsize=50)

y_pred7 = calculate_y_pred_regularized(x_validation, w6, b6, lam=0.1)

acc7 = accuracy(y_validation, y_pred3)
acc7

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9916765755053508

# Testing

In [ ]:
test_data = pd.read_csv('/content/sample_data/mnist_test.csv')

In [ ]:
xtest = test_data[test_data['7'].isin([0, 1])].drop(columns=['7'])

In [ ]:
y_test = test_data[test_data['7'].isin([0, 1])]['7']

In [ ]:
x_test = standardize(xtest)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
x_test, y_test = shuffle(x_test, y_test)

In [ ]:
#L1 regularization lambda 0.1
y_pred11 = calculate_y_pred_regularized(x_test, w1, b1, lam=0.1)

acc11 = accuracy(y_test, y_pred11)
acc11

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9957446808510638

In [ ]:
#L1 regularization lambda 0.3
y_pred12 = calculate_y_pred_regularized(x_test, w2, b2, lam=0.3)

acc12 = accuracy(y_test, y_pred12)
acc12

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9513002364066194

In [ ]:
#mini batch batch size 150
y_pred13 = calculate_y_pred_regularized(x_test, w3, b3, lam=0.1)

acc13 = accuracy(y_test, y_pred13)
acc13

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9976359338061466

In [ ]:
#RMS optimizer
y_pred14 = calculate_y_pred_regularized(x_test, w4, b4, lam=0.1)

acc14 = accuracy(y_test, y_pred14)
acc14

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9976359338061466

In [ ]:
#mini batch batch size 50
y_pred15 = calculate_y_pred_regularized(x_test, w6, b6, lam=0.1)

acc15 = accuracy(y_test, y_pred15)
acc15

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9952718676122931

In [ ]:
#Adam optimizer
y_pred16 = calculate_y_pred_regularized(x_test, w5, b5, lam=0.1)

acc16 = accuracy(y_test, y_pred16)
acc16

<ipython-input-61-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.9905437352245863

---
---

# **Report**

---
---


Report the accuracies for all of the above cases and write a **conclusion** for each case
explaining the behind reasons

# **L1 Regularization**

---


# **L1 Regularization** with gradient descent optimizer:
 first lambda val 0.1

In [ ]:
print(f' lambda = {0.1}\n Accuracy = {acc11} ')

 lambda = 0.1
 Accuracy = 0.9957446808510638 


# **L1 Regularization** with gradient descent optimizer:
 second lambda val 0.3

In [ ]:
print(f' lambda = {0.3}\n Accuracy = {acc12} ')

 lambda = 0.3
 Accuracy = 0.9513002364066194 


**Conclusion** on *L1 Regularization* lambda values:


> Smaller values like 0.1 is better than 0.3 and 0.5 because bigger values decrease the weights values more than needed that some important features have less effect on the data which decreases accuracy.




> Bigger values of lambda helps in decreasing the dependency on less important features and that is useful to avoid overfitting of data.






#**Mini-Batch**

---


# **Mini-Batch** Regularized gradient descent optimizer:
first size =150

In [ ]:
print(f' lambda = {0.1}\n batch size = {150}\n Accuracy = {acc13} ')

 lambda = 0.1
 batch size = 150
 Accuracy = 0.9976359338061466 


# **Mini-Batch** Regularized gradient descent optimizer:
first size =50

In [ ]:
print(f' lambda = {0.1}\n batch size = {50}\n Accuracy = {acc15} ')

 lambda = 0.1
 batch size = 50
 Accuracy = 0.9952718676122931 


**Conclusion** on *Mini-Batch* batch sizes:


> In our case, larger batch sizes makes convergence for weights and the bias slower but more accurate while smaller batch sizes makes convergence for weights and the bias faster but less accurate.




# **Optimizers**

---



# **RMS** Prob optimizer

In [ ]:
print(f' lambda = {0.1}\n Accuracy = {acc14} ')

 lambda = 0.1
 Accuracy = 0.9976359338061466 


# **Adam** optimizer

In [ ]:
print(f' lambda = {0.1}\n Accuracy = {acc16} ')

 lambda = 0.1
 Accuracy = 0.9905437352245863 


**Conclusion** on *Optimizers*:


>In our case, RMS optimizer performed better than Adam optimizer


